So far, we have gathered two types of data. Now, it's time to merge all the collected data and begin the cleaning process.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import requests
from bs4 import BeautifulSoup as bs
import re

In [ ]:
df1 = pd.read_csv('transfer_market.csv')
df2 = pd.read_csv('stats.csv')

In [ ]:
dataF = df1.merge(df2, on = "name")

In [ ]:
# to handle the string values in 'fee' column like 750k and not 750000

def handle_k_values(value):
  try:
    return float(value)
  except ValueError:
    if value[-1] == 'k':
      return float(value[:-1]) * 1000
    else:
      return np.nan

In [ ]:
dataF['fee'] = dataF['fee'].apply(handle_k_values)
dataF['market_value'] = dataF['market_value'].apply(handle_k_values)
dataF['age'] = dataF['age'].astype(int)
dataF['loan_map'] = dataF['loan'].map({False: 0, True: 1})

In [ ]:
dataF.isnull().sum()

Unnamed: 0        0
name              0
position          0
age               0
market_value      0
country_from      0
league_from       0
club_from         0
country_to        0
league_to         0
club_to           0
fee               0
loan              0
ATT             734
SKI             734
MOV             734
POW             734
MEN             734
DEF             734
GK              734
loan_map          0
dtype: int64

In [ ]:
dataF = dataF.dropna()

In [ ]:
dataF = dataF.drop_duplicates()

In [ ]:
dataF = dataF.reset_index(drop=True)
dataF.shape

(708, 21)

In [ ]:
dataF.isnull().sum()

Unnamed: 0      0
name            0
position        0
age             0
market_value    0
country_from    0
league_from     0
club_from       0
country_to      0
league_to       0
club_to         0
fee             0
loan            0
ATT             0
SKI             0
MOV             0
POW             0
MEN             0
DEF             0
GK              0
loan_map        0
dtype: int64

In [ ]:
dataF = dataF.drop(["Unnamed: 0","country_from","country_to"],axis=1)

In [ ]:
colonnes_categorielles_1 = ['league_from', 'league_to']

for colonne in colonnes_categorielles_1:
    counts = dataF[colonne].value_counts()
    categories_autre = counts[counts <4].index.tolist()
    dataF[colonne] = dataF[colonne].apply(lambda x: 'Autre' if x in categories_autre else x)


colonnes_categorielles_2= ['club_to', 'club_from']
for colonne in colonnes_categorielles_2:
    counts = dataF[colonne].value_counts()
    categories_autre = counts[counts < 11].index.tolist()
    if 'Saudi Pro League' in categories_autre:
        categories_autre.remove('Saudi Pro League')

    dataF[colonne] = dataF[colonne].apply(lambda x: 'Autre' if x in categories_autre else x)

In [ ]:
df_encoded = pd.get_dummies(dataF, columns=colonnes_categorielles_1 + colonnes_categorielles_2 + ['position'] + ['loan'], dtype=int)

In [ ]:
df_encoded.shape

(708, 93)

In [ ]:
# To handle string values in the 'fee' column, such as '750k' instead of '750000'

def handle_k_values(value):
  try:
    return float(value)
  except ValueError:
    if value[-1] == 'k':
      return float(value[:-1]) * 1000
    else:
      return np.nan

In [ ]:
df_encoded['fee'] = df_encoded['fee'].apply(handle_k_values)
df_encoded['market_value'] = df_encoded['market_value'].apply(handle_k_values)
df_encoded['age'] = df_encoded['age'].astype(int)

In [ ]:
# Are there two different players with the same name?
# If not, then we can drop duplicates based on the 'name' column
duplicate_rows = df_encoded[df_encoded.duplicated()]
duplicate_rows

,name,age,market_value,fee,ATT,SKI,MOV,POW,MEN,DEF,...,position_Goalkeeper,position_Left Midfield,position_Left Winger,position_Left-Back,position_Right Midfield,position_Right Winger,position_Right-Back,position_Second Striker,loan_False,loan_True
2,Erling Haaland,21,150.0,60.00,76.0,70.0,82.0,86.0,75.0,42.0,...,0,0,0,0,0,0,0,0,1,0
3,Erling Haaland,21,150.0,60.00,60.0,57.0,73.0,66.0,63.0,86.0,...,0,0,0,0,0,0,0,0,1,0
6,Antony,22,35.0,95.00,54.0,63.0,66.0,51.0,49.0,42.0,...,0,0,0,0,0,1,0,0,1,0
7,Antony,22,35.0,95.00,67.0,73.0,71.0,78.0,74.0,81.0,...,0,0,0,0,0,1,0,0,1,0
10,Wesley Fofana,21,40.0,80.40,50.0,54.0,69.0,62.0,62.0,80.0,...,0,0,0,0,0,0,0,0,1,0
11,Wesley Fofana,21,40.0,80.40,72.0,82.0,78.0,73.0,77.0,71.0,...,0,0,0,0,0,0,0,0,1,0
14,Alexander Isak,22,30.0,70.00,71.0,78.0,92.0,72.0,68.0,49.0,...,0,0,0,0,0,0,0,0,1,0
15,Alexander Isak,22,30.0,70.00,76.0,77.0,76.0,75.0,70.0,39.0,...,0,0,0,0,0,0,0,0,1,0
18,Matthijs de Ligt,22,70.0,67.00,81.0,75.0,71.0,80.0,72.0,34.0,...,0,0,0,0,0,0,0,0,1,0
19,Matthijs de Ligt,22,70.0,67.00,51.0,66.0,70.0,73.0,67.0,82.0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
df_encoded.drop_duplicates(subset='name', inplace=True)
df_encoded.dropna(inplace=True)

In [ ]:
df_encoded = df_encoded.reset_index(drop=True)

In [ ]:
df_encoded.head()

,name,age,market_value,fee,ATT,SKI,MOV,POW,MEN,DEF,...,position_Goalkeeper,position_Left Midfield,position_Left Winger,position_Left-Back,position_Right Midfield,position_Right Winger,position_Right-Back,position_Second Striker,loan_False,loan_True
0,Erling Haaland,21,150.0,60.0,76.0,70.0,82.0,86.0,75.0,42.0,...,0,0,0,0,0,0,0,0,1,0
1,Antony,22,35.0,95.0,54.0,63.0,66.0,51.0,49.0,42.0,...,0,0,0,0,0,1,0,0,1,0
2,Wesley Fofana,21,40.0,80.4,50.0,54.0,69.0,62.0,62.0,80.0,...,0,0,0,0,0,0,0,0,1,0
3,Alexander Isak,22,30.0,70.0,71.0,78.0,92.0,72.0,68.0,49.0,...,0,0,0,0,0,0,0,0,1,0
4,Matthijs de Ligt,22,70.0,67.0,81.0,75.0,71.0,80.0,72.0,34.0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613 entries, 0 to 612
Data columns (total 93 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   name                              613 non-null    object 
 1   age                               613 non-null    int64  
 2   market_value                      613 non-null    float64
 3   fee                               613 non-null    float64
 4   ATT                               613 non-null    float64
 5   SKI                               613 non-null    float64
 6   MOV                               613 non-null    float64
 7   POW                               613 non-null    float64
 8   MEN                               613 non-null    float64
 9   DEF                               613 non-null    float64
 10  GK                                613 non-null    float64
 11  loan_map                          613 non-null    int64  
 12  league_f

In [ ]:
df_encoded.describe()

,age,market_value,fee,ATT,SKI,MOV,POW,MEN,DEF,GK,...,position_Goalkeeper,position_Left Midfield,position_Left Winger,position_Left-Back,position_Right Midfield,position_Right Winger,position_Right-Back,position_Second Striker,loan_False,loan_True
count,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,...,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000
mean,24.760196,5147.741599,15088.775628,58.154976,59.942904,68.525285,65.657423,59.226754,50.858075,13.678630,...,0.057096,0.006525,0.096248,0.060359,0.008157,0.070147,0.055465,0.008157,0.734095,0.265905
std,3.659920,60448.434379,85296.806604,12.722932,13.156611,9.738313,8.751663,10.017915,21.385073,14.192289,...,0.232216,0.080581,0.295172,0.238345,0.090018,0.255603,0.229072,0.090018,0.442175,0.442175
min,18.000000,0.000000,0.000000,10.000000,13.000000,29.000000,32.000000,17.000000,9.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.000000,3.500000,0.000000,54.000000,55.000000,64.000000,61.000000,55.000000,30.000000,9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,24.000000,5.000000,3.000000,61.000000,62.000000,70.000000,67.000000,61.000000,58.000000,11.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,27.000000,10.000000,7.100000,66.000000,69.000000,75.000000,72.000000,66.000000,70.000000,12.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,35.000000,900000.000000,890000.000000,86.000000,88.000000,92.000000,86.000000,81.000000,86.000000,80.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [69]:
df_encoded.to_csv('model_training_data.csv')